In [17]:
%pip install transformers
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import numpy as np
import pandas as pd
import transformers
import torch
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [19]:
test_data = pd.read_csv("test.csv")

In [20]:
test_data.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318


In [21]:
train_data = pd.read_csv("train.csv")

In [22]:
train_data.head()

,Unnamed: 0,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650.0,2125.980000
1,1,2885531,STORE99® Acrylic Crystal Butterfly Beads Curta...,"[Good quality and Suitable to use.,This Produc...",STORE99 Acrylic Crystal Butterfly Beads Curtai...,1650.0,590.551180
2,2,2388637,COZY FURNISH Linen Textured Cotton Curtains fo...,[WHY COZY FURNISH CURTAINS: Our premium cotton...,WHY COZY FURNISH CURTAINS: Our premium cotton ...,1650.0,39.370079
3,3,2855853,3NH® Hotel Decoration Window Thickened Yarn Co...,"[Good quality and Suitable to use.,This Produc...",3NH Hotel Decoration Window Thickened Yarn Cor...,1650.0,590.551180
4,4,1776083,Home Decor Kart 6Ft Curtains for Window Set of...,"[Transparency :Semi transparent, Hang Type:Eye...",This curtain enhances the look of the interior...,1650.0,6000.000000


In [23]:
train_data = train_data.sample(n=10000, random_state=123)

In [24]:
train_data = train_data[train_data['PRODUCT_LENGTH'] >= 0]
train_data['PRODUCT_LENGTH'] = np.log1p(train_data['PRODUCT_LENGTH'])

In [25]:
train_set, val_set = train_test_split(train_data, test_size=0.2, random_state=123)

In [26]:
vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(train_set['TITLE'].fillna('') + ' ' + train_set['DESCRIPTION'].fillna('') + ' ' + train_set['BULLET_POINTS'].fillna(''))
val_features = vectorizer.transform(val_set['TITLE'].fillna('') + ' ' + val_set['DESCRIPTION'].fillna('') + ' ' + val_set['BULLET_POINTS'].fillna(''))

In [27]:
model = LinearRegression()
model.fit(train_features, train_set['PRODUCT_LENGTH'].fillna(0))

LinearRegression()

In [28]:
val_predictions = model.predict(val_features)

In [29]:
from sklearn.metrics import mean_squared_error, r2_score

In [30]:
mse = mean_squared_error(val_set['PRODUCT_LENGTH'], val_predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.45363167619858685


In [31]:
test_features = vectorizer.transform(test_data['TITLE'].fillna('') + ' ' + test_data['DESCRIPTION'].fillna('') + ' ' + test_data['BULLET_POINTS'].fillna(''))
test_predictions = model.predict(test_features)
submission_df = pd.DataFrame({'PRODUCT_ID': test_data['PRODUCT_ID'], 'PRODUCT_LENGTH': test_predictions})

In [33]:
submission_df.to_csv("submission2.csv", index=False)

In [34]:
submission_df.shape

(734736, 2)